In [ ]:
import numpy as np
import scipy as sp
import tensorflow as tf
from datetime import date
from datetime import datetime
from datetime import timedelta
import calendar
import pickle
from scipy.sparse.linalg import LinearOperator
import os.path
import time
import sys
# you need to install the trlib and change the path to the path you installed the trlib
# trlib can be found in  https://github.com/felixlen/trlib
sys.path.insert(0, '/home/knian/trlib-master/')
import trlib
from tensorflow.python import debug as tf_debug
# tensorflow_forward_ad can be found in https://pypi.org/project/tensorflow_forward_ad/
from tensorflow_forward_ad.second_order import hessian_vec_bk
import gc
from sklearn.model_selection import KFold

In [ ]:
class Basic_Model(object):
    def __init__(self, inp_dim, inp_seq, inp_dim_y, state_dim, state_output):
        # Define placeholders for sequential input, tabular input, and output
        self.x_t = tf.placeholder(tf.float32, [inp_seq,None,inp_dim])
        self.y = tf.placeholder(tf.float32, [None,inp_dim_y])
        self.z = tf.placeholder(tf.float32, [None,3])
        self.cs = tf.placeholder(tf.float32, [None])
        self.cv = tf.placeholder(tf.float32, [None])
        self.bs = tf.placeholder(tf.float32, [None])
        # setting huber threshold as the placeholder if you want to set manual threshold you can set it.
        # in paper, huber threshold is set to be the hedging error of bs hedging
        self.threshold = tf.placeholder(tf.float32, shape=())
        #setting the dropout placeholder and penalty for parameter norm which is used to regularized the network when it is large.
        self.keep_prob = tf.placeholder(tf.float32, shape=())
        self.penalty = tf.placeholder(tf.float32, shape=())
        # create the large parameters vector
        orthogonal=tf.orthogonal_initializer()
        parameters = tf.Variable(tf.concat( [
                                             #gated recurrent unit
                                             tf.truncated_normal([state_dim * inp_dim*3, 1]),
                                             tf.expand_dims(tf.reshape(orthogonal([state_dim , state_dim]),[-1]),1),
                                             tf.expand_dims(tf.reshape(orthogonal([state_dim , state_dim]),[-1]),1),
                                             tf.expand_dims(tf.reshape(orthogonal([state_dim , state_dim]),[-1]),1),
                                             tf.truncated_normal([3*state_dim, 1]),
                                             #NN decoder
                                             tf.truncated_normal([state_dim*state_output, 1]),
                                             tf.truncated_normal([state_output*(inp_dim_y+1), 1]),
                                             tf.truncated_normal([state_output, 1]),
                                             tf.truncated_normal([state_output, 1]),
                                             #Final Gate to Determine Whether to use BS or not
                                             tf.truncated_normal([state_output*state_dim, 1]),
                                             tf.truncated_normal([state_output*(inp_dim_y+1), 1]),
                                             tf.truncated_normal([state_output, 1]),
                                             tf.truncated_normal([state_output, 1]),
                                             # weighting
                                             tf.ones([inp_dim_y+1, 1]),
                                             tf.ones([inp_dim, 1]),
                                             # MV model   
                                             tf.truncated_normal([3, 1])],0))
        
        self.par=parameters
        self.vec = tf.placeholder(tf.float32, shape=[parameters.get_shape()[0],1], name='vec')
        self.update = tf.placeholder(tf.float32, shape=[parameters.get_shape()[0],1], name='update')
        print('size of parameters', self.par.get_shape())
        # placeholder for the initial hidden states
        self.c=tf.placeholder(tf.float32, [None,state_dim])
        self.h=tf.placeholder(tf.float32, [None,state_dim])
        idx_from = 0     
        #Define the GRU encoder weights
        with tf.name_scope("GRU") as scope:
            self.U = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_dim*inp_dim*3, 1]), [3, inp_dim,state_dim])
            idx_from = idx_from + state_dim*inp_dim*3
            self.W = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_dim*state_dim*3, 1]), [3,state_dim, state_dim])
            idx_from = idx_from + state_dim*state_dim*3
            self.b = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_dim*3, 1]), [3,state_dim])
            idx_from = idx_from + state_dim*3
        
        with tf.name_scope("Decoder") as scope:
            self.Uout = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_dim*state_output, 1]), [state_dim, state_output])
            idx_from = idx_from + state_dim*state_output
            self.Wout = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output*(inp_dim_y+1), 1]), [inp_dim_y+1,state_output]) 
            idx_from = idx_from + state_output*(inp_dim_y+1)
            self.bout = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output, 1]), [state_output]) 
            idx_from = idx_from + state_output
            self.Vout = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output, 1]), [state_output,1])
            idx_from = idx_from + state_output
            
        with tf.name_scope("FinalGates") as scope:
            self.Ugate = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output*state_dim, 1]), [state_dim,state_output])
            idx_from = idx_from + state_output*state_dim
            self.Wgate = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output*(inp_dim_y+1), 1]), [(inp_dim_y+1),state_output])
            idx_from = idx_from + state_output*(inp_dim_y+1)
            
            self.bgate = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output, 1]), [state_output])
            idx_from = idx_from + state_output
            self.Vgate = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[state_output, 1]), [state_output,1])
            idx_from = idx_from + state_output
       
        
        with tf.name_scope("weighting") as scope:
            self.FW = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[inp_dim_y+1, 1]), [inp_dim_y+1])
            idx_from = idx_from + inp_dim_y+1
            self.SW = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[inp_dim, 1]), [inp_dim])
            idx_from = idx_from + inp_dim
        with tf.name_scope("MV") as scope:
            self.MVa = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[1, 1]), [1])
            idx_from = idx_from + 1
            self.MVb = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[1, 1]), [1])
            idx_from = idx_from + 1
            self.MVc = tf.reshape(tf.slice(parameters, begin=[idx_from, 0], size=[1, 1]), [1])
            idx_from = idx_from + 1
       
            

        def forward_step_GRU(acc, x_i):
            h_p = acc
            x_hat=tf.matmul(x_i,self.U[0])
            # apply DropOut to input layer
            x_hat = tf.nn.dropout(x_hat, self.keep_prob) 
            h_hat=tf.matmul(h_p,self.W[0])
            z_t = tf.nn.sigmoid(tf.add(x_hat+h_hat, self.b[0]))
            x_hat=tf.matmul(x_i,self.U[1])
            h_hat=tf.matmul(h_p,self.W[1])
            x_hat = tf.nn.dropout(x_hat, self.keep_prob) 
            r_t = tf.nn.sigmoid(tf.add(x_hat + h_hat, self.b[1]))
            x_hat=tf.matmul(x_i,self.U[2])
            h_c=tf.multiply(r_t,h_p)
            h_hat=tf.matmul(h_c,self.W[2])
            x_hat = tf.nn.dropout(x_hat, self.keep_prob) 
            o_t = tf.nn.tanh(tf.add(x_hat + h_hat, self.b[2]))
            h= tf.multiply(1-z_t,h_p) + tf.multiply(z_t,o_t)
            return h
        
        
        # prepare the local input ans weight the local input
        MV=tf.multiply(self.MVa,self.z[:,0])+tf.multiply(self.MVb,self.z[:,1])+tf.multiply(self.MVc,self.z[:,2])+self.bs
        y_W=tf.nn.softmax(self.FW)
        w_y=np.multiply(self.y,y_W[0:-1])
        WMV=np.multiply(MV,y_W[-1])
        y_concat = tf.concat([tf.expand_dims(WMV,1),w_y], axis = 1)
        #weight the sequential input
        S_W=tf.nn.softmax(self.SW)
        W_feature_k=[]
        for k in range(inp_dim):
            W_feature_k.append(tf.expand_dims(tf.multiply(self.x_t[:,:,k],S_W[k]),2))
        w_x_t=tf.concat(W_feature_k,axis=2)
        print('w_x_t:',w_x_t.get_shape())
        
        # Step through the sequence of input words, each one at a time using weighted input X
        h_init = self.h
        hiddenlist=[]
        hiddenlist.append(h_init)
        for i in range(inp_seq):
            h = forward_step_GRU(hiddenlist[-1],w_x_t[i,:,:])
            hiddenlist.append(h)
        # the hidden states from last step is used in decoder
        hE=hiddenlist[-1]
        


        hE_hat=tf.matmul(hE,self.Ugate)
        y_hat=tf.matmul(y_concat,self.Wgate)
        Gate_hat = tf.nn.dropout(hE_hat, self.keep_prob)+tf.nn.dropout(y_hat, self.keep_prob)+self.bgate
        GateV=tf.nn.sigmoid(tf.matmul(tf.tanh(Gate_hat),self.Vgate))
        y_hat=tf.matmul(y_concat,self.Wout)
        hE_hat=tf.matmul(hE,self.Uout)
        y_hat = tf.nn.dropout(y_hat, self.keep_prob)
        hE_hat = tf.nn.dropout(hE_hat, self.keep_prob)
        finalY=y_hat+hE_hat+self.bout
        predNN=tf.matmul(tf.tanh(finalY),self.Vout)
        
        
        pred=tf.nn.sigmoid(predNN)
        pred=tf.multiply(GateV,tf.expand_dims(self.bs,1))+tf.multiply(1-GateV,pred)
        
        pred=tf.squeeze(pred,axis=[-1])
        self.pred=pred
        self.y_concat=y_concat
        self.hE=hE
        self.y_W=y_W
        self.S_W=S_W
        self.hedge_Loss=tf.subtract(tf.multiply(pred,self.cs),self.cv)
        self.hedge_Loss_bs=tf.subtract(tf.multiply(self.bs,self.cs),self.cv)
        cost = tf.square(self.hedge_Loss)
        costabs = tf.abs(self.hedge_Loss)
        
        
        threshold=self.threshold
        penalty=self.penalty
        
        penalized_par=tf.concat([tf.reshape(self.U,[-1]),tf.reshape(self.Uout,[-1]),
         tf.reshape(self.Wout,[-1]),tf.reshape(self.Vout,[-1]),
         tf.reshape(self.bout,[-1]),tf.reshape(self.Vgate,[-1]),tf.reshape(self.Ugate,[-1]),
         tf.reshape(self.Wgate,[-1]),tf.reshape(self.bgate,[-1])],axis=0)
        print('P_Par:', penalized_par.get_shape())
        N=penalized_par.shape[0].value
        penalized_norm=np.divide(tf.square(tf.norm(penalized_par)),N)
          
        
        BScost = tf.square(self.hedge_Loss_bs)
        
        #alternative huber loss with threshold
#         condition = tf.less(tf.abs(self.hedge_Loss), self.threshold)
#         small_res = 0.5*tf.square(self.hedge_Loss)
#        large_res = tf.multiply( self.threshold,tf.abs(self.hedge_Loss_bs))-0.5*tf.square(self.hedge_Loss_bs)
#         costhuber= tf.where(condition, small_res, large_res)
        
        #huber loss
        condition = tf.less(tf.abs(self.hedge_Loss), tf.abs(self.hedge_Loss_bs))
        small_res = 0.5*tf.square(self.hedge_Loss)
        large_res = tf.multiply(tf.abs(self.hedge_Loss),tf.abs(self.hedge_Loss_bs))-0.5*tf.square(self.hedge_Loss_bs)
        costhuber= tf.where(condition, small_res, large_res)
        
        
        #loss to push model to output bs initially
        condition = tf.less(tf.abs(self.hedge_Loss), tf.abs(self.hedge_Loss_bs))
        large_res = tf.abs(self.hedge_Loss_bs-self.hedge_Loss)
        small_res = tf.abs(self.hedge_Loss_bs)
        costnonC= tf.where(condition, small_res, large_res)
        
        
        
        self.total_losshuber = tf.reduce_mean(costhuber)+self.penalty*penalized_norm
        self.total_lossnonC = tf.reduce_mean(costnonC)
        costMV = tf.square(tf.subtract(tf.multiply(MV,self.cs),self.cv))
        BScost = tf.square(self.hedge_Loss_bs)
        self.total_loss = tf.reduce_sum(cost)
        self.total_lossabs = tf.reduce_sum(costabs)
        self.total_loss_BS = tf.reduce_sum(BScost)
        self.total_lossmv = tf.reduce_sum(costMV)
        self.radius=1.0
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_global_step = self.global_step.assign(0)
        self.learning_rate = tf.train.exponential_decay(0.01, self.global_step, 1000, 0.96, staircase=True)
        self.learning_adam = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.total_losshuber, global_step=self.global_step)
        self.learning_adamNC = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.total_lossnonC, global_step=self.global_step)
        self.grad=tf.gradients(self.total_losshuber,parameters)[0]
        self.gradNC=tf.gradients(self.total_lossnonC,parameters)[0]
        self.Hv = hessian_vec_bk(self.total_losshuber, parameters, self.vec,grads=self.grad)
        self.load_par = self.par.assign(self.update)
        
        
    def TrustRegionUpdate(self,feedH,feedG,maxRadius):
        t0 = time.time()
        p,approxloss=self.TRSTrlib(self.radius,feedH,feedG)
        t1 = time.time()
        total = t1-t0
        #print('TRS takes:',total)
        p=p.real
        curloss=self.sess.run(self.total_losshuber, feed_dict=feedG)
        curpar=self.sess.run(self.par,feed_dict=feedG)
        trialpar=curpar+p
        self.sess.run(self.load_par, feed_dict={self.update:trialpar})
        trialloss=self.sess.run(self.total_losshuber, feed_dict=feedG)
        rho=(trialloss-curloss)/(approxloss)
        eta = 0.1 
        mu = 0 
        gamma0 = 0.5 
        gamma1 = 0.9
        gamma2=1.1
        
        pnorm=np.linalg.norm(p)
        if rho<0:
            gammaK = gamma0
            self.radius = min(self.radius*gammaK,maxRadius)
            #print('shrink radius and revert the changes because rho<=0')
            self.sess.run(self.load_par, feed_dict={self.update:curpar})
        else:
            if rho<mu:
                #print('shrink radius and revert the changes because rho<=mu')
                self.sess.run(self.load_par, feed_dict={self.update:curpar})
                gammaK = max(gamma0,gamma1*pnorm/self.radius)
                self.radius = self.radius*gammaK
            else:
                #print('accept trials')          
                if rho>=eta:
                    #print('enlarge radius')
                    gammaK=gamma2
                    self.radius = min(self.radius*gammaK,maxRadius)

    def TRSTrlib(self,radius,feedH,feedG):
        def HVFun(v,feed):
            feed[model.vec]=np.expand_dims(v,axis=1)
            HV=self.sess.run(model.Hv,feed_dict=feed)
            HV=np.squeeze(HV)
            return HV
        g=self.sess.run(self.grad,feed_dict=feedG)
        g=np.squeeze(g)
        HVOps= lambda v: HVFun(v,feedH)
        H = LinearOperator((g.shape[0],g.shape[0]), matvec=HVOps, rmatvec=HVOps)
        p, TR = trlib.trlib_solve(H, g, radius,verbose=0)
        #print('finish solving sub')
        return np.expand_dims(p,axis=1),TR['obj']

In [ ]:
#initialized the model
tf.reset_default_graph()
model=Basic_Model(inp_dim=6, inp_seq=6, inp_dim_y=9, state_dim=5, state_output=5)
# we are not allowed to share the data but here is the what you need to provide.
# XMat: sequential input
# YMat: local input
# ZMat: The three dimension needed for Hull-White MV model
# CSMat: the vector of changes of stock prices
# CVMat: the vector of changes of option prices
# h: initial state value for GRU encoder (usually set to be zero)
# prepare the feed dict
feedMat = { model.x_t:XMat,model.y:YMat,model.z:ZMat,model.cs:CSMat,model.cv:CVMat,model.bs:BSMat,model.h:h}
feedVal = { model.x_t:XVal,model.y:YVal,model.z:ZVal,model.cs:CSVal,model.cv:CVVal,model.bs:BSVal,model.h:hv}
feedTest = { model.x_t:XTest,model.y:YTest,model.z:ZTest,model.cs:CSTest,model.cv:CVTest,model.bs:BSTest,model.h:ht}
# prepare the feed dict, in the paper, we do not regularized the model parameter since the model is small. 
# You will need regularization if your model is getting larger.
feedMat[model.penalty]=0.0
feedVal[model.penalty]=0.0
feedTest[model.penalty]=0.0
feedMat[model.keep_prob]=1.0
feedVal[model.keep_prob]=1.0
feedTest[model.keep_prob]=1.0

In [ ]:
#start the model session    
model.sess=tf.train.MonitoredTrainingSession(
                                        save_checkpoint_secs=None,
                                        save_summaries_steps=None,
                                        save_summaries_secs=None)

In [ ]:
#using adam to pre-training the model, if converged, we skip the trust region update    
model.sess.run(model.initial_global_step)
ConvergeFlag=False    
for i in range(5000):
    model.sess.run([model.learning_adam],feed_dict = feedMat)
    grad=model.sess.run(model.grad,feed_dict=feedMat)
    loss,loss_bs= model.sess.run([model.total_loss,model.total_loss_BS],feed_dict = feedMat)
    lossV,loss_bsV= model.sess.run([model.total_loss,model.total_loss_BS],feed_dict = feedVal)
    lossT,loss_bsT= model.sess.run([model.total_loss,model.total_loss_BS],feed_dict = feedTest)
    if i%10==0:
        print('Adam Step',i,'Date',CurDate)
        print('Train Gain',1-loss/loss_bs)
        print('Val Gain',1-lossV/loss_bsV)
        print('Test Gain',1-lossT/loss_bsT)
        print('Grad Norm',np.linalg.norm(grad))
    CurVal=1-lossV/loss_bsV
    CurTest=1-lossT/loss_bsT

    #update the best val error
    if CurVal>OverAllBest:
        Par= model.sess.run([model.par])
        OverAllBestPred= model.sess.run([model.pred],feed_dict = feedTest)
        OverAllBestPar=(Par,OverAllBestPred,BSTest,CSTest,CVTest)
        OverAllBest=CurVal
        BestTest=CurTest
    #update the best test error
    if CurTest>BestVal:
        Par= model.sess.run([model.par])
        BestPred= model.sess.run([model.pred],feed_dict = feedTest)
        BestPar=(Par,BestPred,BSTest,CSTest,CVTest)
        BestVal=CurTest
    if np.linalg.norm(grad)<=0.001:
        print('get a critical point, break iteration')
        print('Adam Step',i,'Date',CurDate)
        print('Train Gain',1-loss/loss_bs)
        print('Val Gain',1-lossV/loss_bsV)
        print('Test Gain',1-lossT/loss_bsT)
        print('Grad Norm',np.linalg.norm(grad))
        print('Current Best Val ', OverAllBest,'Best Test',BestTest)
        print('Actual Best Val ', BestVal)
        ConvergeFlag=True
        break



In [ ]:
#using adam to pre-training the model, if converged, we skip the trust region update    
for i in range(MaxIter):
    model.TrustRegionUpdate(feedMat,feedMat,1000000.0)  
    print('Trust Region Step',i)
    print('Train Gain',1-loss/loss_bs)
    print('Val Gain',1-lossV/loss_bsV)
    print('Test Gain',1-lossT/loss_bsT)
    print('Grad Norm',np.linalg.norm(grad))
    print('Radius',model.radius)
    CurVal=1-lossV/loss_bsV    
    CurTest=1-lossT/loss_bsT
    if model.radius<=10**-5:
        print('model radius too small reset to 1.0, break iteration')
        print('Trust Region Step',i,'Date',CurDate)
        print('Train Gain',1-loss/loss_bs)
        print('Val Gain',1-lossV/loss_bsV)
        print('Test Gain',1-lossT/loss_bsT)
        print('Grad Norm',np.linalg.norm(grad))
        print('Radius',model.radius)
        break
    if np.linalg.norm(grad)<=0.001:
        print('get a critical point, break iteration')
        print('Trust Region Step',i,'Date',CurDate)
        print('Train Gain',1-loss/loss_bs)
        print('Val Gain',1-lossV/loss_bsV)
        print('Test Gain',1-lossT/loss_bsT)
        print('Grad Norm',np.linalg.norm(grad))
        print('Radius',model.radius)
        break
